In [1]:
import networkx as nx
import scipy as sp
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA

In [8]:
def get_polarization_index(G):

    #opinion of each node
    #the S array

    opinion_values = nx.get_node_attributes(G,'value')
    opinion_values = np.array(list(opinion_values.values()))

    #size of the graph 
    size = opinion_values.shape[0]
    #format them as a N x 1 array for the multiplication
    opinion_values = opinion_values.reshape(size,1)

    #the laplacian matrix of the graph
    #return scipy sparse matrix, convert it to
    #numpy array with .toarray()
    L = nx.laplacian_matrix(G).toarray()

    #identity matrix of the same size of the laplacian

    I = np.identity(size)

    # Add two matrices - two nd arrays

    add_results = L.__add__(I)

    # compute the fundamental matrix 
    # Q = (L+I)^-1

    Q = inv(add_results)

    # the opinion vector of the network

    Z = add_results * opinion_values

    # the polarization index

    polarization_index =  LA.norm(Z, 2) / size

    print("Polarization of the graph:",polarization_index)


### original graph

In [9]:
karate=nx.read_gml("karate.gml",label='id')

#opinions vary from 0 to 1, find all zero occurences
zero_indices = [k for (k, v) in value_dictionary.items() if v == 0]

#empty dictionary
attrs = {}

#create the dictionary that will update 0 nodes to -1
for obj in zero_indices:
    d = {obj:{'value':-1}}
    attrs.update(d)

#se the new opinion values
nx.set_node_attributes(karate, attrs)

#get the values of the new graph in a dictionary
value_dictionary = nx.get_node_attributes(karate,'value')

print(nx.info(karate))

get_polarization_index(karate)

Name: 
Type: Graph
Number of nodes: 34
Number of edges: 78
Average degree:   4.5882
Polarization of the graph: 0.5628439992060117


### connecting negative nodes with each other

In [10]:
negative_graph = karate.copy()

negative_indices = [k for (k, v) in value_dictionary.items() if v == -1]

all_pairs = []
pairs = []
edges_to_add = []

for node in value_dictionary:

    if(value_dictionary[node] == -1):
        
        current_node_list_for_pairing = [node]
        #make unique pairs from two lists for adding edges
        pairs = [(a, b) for a in current_node_list_for_pairing for b in negative_indices if a != b] 
        
        #add all pairs that were created
        all_pairs.extend(pairs) 
        


#clean duplicate edges, [a,b]==[b,a]

edges_to_add = list({tuple(sorted(item)) for item in all_pairs})

#adding these edges
negative_graph.add_edges_from(edges_to_add)
    
print(nx.info(negative_graph))

get_polarization_index(negative_graph)


Name: 
Type: Graph
Number of nodes: 34
Number of edges: 179
Average degree:  10.5294
Polarization of the graph: 0.6516771884964171


### connecting positive nodes with each other

In [11]:
positive_graph = karate.copy()


positive_indices = [k for (k, v) in value_dictionary.items() if v == 1]

all_pairs = []
pairs = []
edges_to_add = []

for node in value_dictionary:
    
    if(value_dictionary[node] == 1):
        
        current_node_list_for_pairing = [node]

        #make unique pairs from two lists for adding edges
        pairs = [(a, b) for a in current_node_list_for_pairing for b in positive_indices if a != b] 
        
        #add all pairs that were created
        all_pairs.extend(pairs) 


#clean duplicate edges, [a,b]==[b,a]

edges_to_add = list({tuple(sorted(item)) for item in all_pairs})
        
#adding these edges
positive_graph.add_edges_from(edges_to_add)
    
print(nx.info(positive_graph))

get_polarization_index(positive_graph)

Name: 
Type: Graph
Number of nodes: 34
Number of edges: 182
Average degree:  10.7059
Polarization of the graph: 0.6199455697613875
